# Visualize Evaluation Run

In [ ]:
# @title Configuration
# fmt: off
GOOGLE_CLOUD_PROJECT = "" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
GOOGLE_CLOUD_REGION = "" # @param {type: "string", placeholder: "[your-region]", isTemplate: true}
EXPERIMENT_NAME = "agent-evaluation-researcher" # @param {type: "string", placeholder: "[experiment name]", isTemplate: true}
RUN_NAME = "(latest)" # @param {type: "string", placeholder: "[run name]", isTemplate: true}
# fmt: on

## Dependencies

In [ ]:
%pip install google-cloud-aiplatform[evaluation]==1.133.* vertexai google-genai==1.59.* -q

## Authentication for Google Colab

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Initialize

In [ ]:
import os
from google.cloud import aiplatform
from dotenv import load_dotenv

load_dotenv()

if not GOOGLE_CLOUD_PROJECT:
    GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT")

if not GOOGLE_CLOUD_REGION:
    GOOGLE_CLOUD_REGION = os.getenv(
        "GOOGLE_CLOUD_REGION",
        os.getenv(
            "GOOGLE_CLOUD_LOCATION",
        ),
)

project_id = GOOGLE_CLOUD_PROJECT
location = GOOGLE_CLOUD_REGION
if not location or location == "global":
   location = "us-central1"

aiplatform.init(project=project_id, location=location)

## Retrieve Evaluation Run

In [ ]:
# TODO

## Show Evaluation Report

In [ ]:
# TODO